# Задача: 

- убрать по частоте (встретилось не более 3х раз)
- убрать по pymorphy (FakeDictionary)

In [1]:
import json

In [2]:
with open('./all_data.json', 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [6]:
word = 'инфор\xadмация'

In [7]:
p = morph.parse(word)

In [8]:
p

[Parse(word='инфор\xadмация', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='инфор\xadмация', score=1.0, methods_stack=((<FakeDictionary>, 'инфор\xadмация', 40, 0), (<KnownSuffixAnalyzer>, 'мация')))]

In [9]:
methods_stack = p[0].methods_stack[0][0]

In [10]:
p2 = morph.parse('самозатухание')

In [11]:
p2[0].methods_stack[0][0] is methods_stack

False

In [12]:
p[0].methods_stack[0][0] is methods_stack

True

Логичнее всего начать "снизу вверх":

- убираем низкочастотные существительные
- убираем существительные с ошибками
- убираем низкочастотные предлоги
- убираем предлоги с ошибками
- убираем низкочастотные глаголы
- убираем глаголы с ошибками

Замеряем: 

- количество уникальных существительных "до" и "после"
- количество уникальных предлогов "до" и "после"
- количество уникальных глаголов "до" и "после"
- количество всех сочетаний "до" и "после"

In [10]:
def count_stat(a, count=None):
    if count is None: count = 0
    for key in a:
        if isinstance(a[key], dict):
            count = count_stat(a[key], count)
        else:
            count += a[key]
    return count

In [14]:
print('Количество сочетаний "до":', count_stat(json_data))

Количество сочетаний "до": 37841081


In [17]:
unique_verbs = set(json_data["_"])
unique_verbs.update(json_data["не"])
print('Количество уникальных глаголов "до":', len(unique_verbs))

Количество уникальных глаголов "до": 66462


In [18]:
unique_adps = set()
for part in json_data:
    for verb in json_data[part]:
        unique_adps.update(json_data[part][verb])
print('Количество уникальных предлогов "до":', len(unique_adps))

Количество уникальных предлогов "до": 2951


In [20]:
unique_nouns = set()
for part in json_data:
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                unique_nouns.update(json_data[part][verb][adp][case])
print('Количество уникальных существительных "до":', len(unique_nouns))

Количество уникальных существительных "до": 196128


Фильтрация существительных

In [21]:
noun_dict = dict()
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                for noun in json_data[part][verb][adp][case]:
                    if noun not in noun_dict:
                        noun_dict[noun] = json_data[part][verb][adp][case][noun]
                    else:
                        noun_dict[noun] += json_data[part][verb][adp][case][noun]

In [23]:
noun_for_delete = set()
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                noun_list = list()
                for noun in json_data[part][verb][adp][case]:
                    if noun_dict[noun] <= 3:
                        noun_list.append(noun)
                    else:
                        p = morph.parse(noun)
                        if p[0].methods_stack[0][0] is methods_stack:
                            noun_list.append(noun)
                for noun in noun_list:
                    json_data[part][verb][adp][case].pop(noun, None)
                    noun_for_delete.add(noun)
print('Количество удаленных существительных:', len(noun_for_delete))

Количество удаленных существительных: 147731


In [24]:
unique_nouns = set()
for part in json_data:
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                unique_nouns.update(json_data[part][verb][adp][case])
print('Количество уникальных существительных "после":', len(unique_nouns))

Количество уникальных существительных "после": 48397


In [25]:
print('Количество сочетаний "после":', count_stat(json_data))

Количество сочетаний "после": 37310743


Фильтрация предлогов

In [26]:
adp_dict = dict()
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            if adp not in adp_dict:
                adp_dict[adp] = count_stat(json_data[part][verb][adp])
            else:
                adp_dict[adp] += count_stat(json_data[part][verb][adp])

In [29]:
adp_for_delete = set()
for part in json_data: 
    for verb in json_data[part]:
        adp_list = list()
        for adp in json_data[part][verb]:
            if adp_dict[adp] <= 3:
                adp_list.append(adp)
            else:
                p = morph.parse(adp)
                if p[0].methods_stack[0][0] is methods_stack:
                    adp_list.append(adp)
        for adp in adp_list:
            json_data[part][verb].pop(adp, None)
            adp_for_delete.add(adp)
print('Количество удаленных предлогов:', len(adp_for_delete))

Количество удаленных предлогов: 2680


In [31]:
unique_adps = set()
for part in json_data:
    for verb in json_data[part]:
        unique_adps.update(json_data[part][verb])
print('Количество уникальных предлогов "после":', len(unique_adps))

Количество уникальных предлогов "после": 271


In [32]:
print('Количество сочетаний "после":', count_stat(json_data))

Количество сочетаний "после": 37307016


Фильтрация глаголов

In [33]:
verb_dict = dict()
for part in json_data: 
    for verb in json_data[part]:
        if verb not in verb_dict:
            verb_dict[verb] = count_stat(json_data[part][verb])
        else:
            verb_dict[verb] += count_stat(json_data[part][verb])

In [35]:
verb_for_delete = set()
for part in json_data: 
    verb_list = list()
    for verb in json_data[part]:
        if verb_dict[verb] <= 3:
            verb_list.append(verb)
        else:
            p = morph.parse(verb)
            if p[0].methods_stack[0][0] is methods_stack:
                verb_list.append(verb)
    for verb in verb_list:
        json_data[part].pop(verb, None)
        verb_for_delete.add(verb)
print('Количество удаленных глаголов:', len(verb_for_delete))

Количество удаленных глаголов: 44682


In [36]:
unique_verbs = set(json_data["_"])
unique_verbs.update(json_data["не"])
print('Количество уникальных глаголов "после":', len(unique_verbs))

Количество уникальных глаголов "после": 21780


In [37]:
print('Количество сочетаний "после":', count_stat(json_data))

Количество сочетаний "после": 36792942


In [38]:
unique_adps = set()
for part in json_data:
    for verb in json_data[part]:
        unique_adps.update(json_data[part][verb])
print('Количество уникальных предлогов "после":', len(unique_adps))

Количество уникальных предлогов "после": 270


In [39]:
unique_nouns = set()
for part in json_data:
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                unique_nouns.update(json_data[part][verb][adp][case])
print('Количество уникальных существительных "после":', len(unique_nouns))

Количество уникальных существительных "после": 48380


In [40]:
with open('./all_data_after_filter.json', 'w', encoding='UTF-8') as f:
    json.dump(json_data, f)

Удаление именительного падежа!

In [41]:
for part in json_data: 
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            json_data[part][verb][adp].pop('Nom', None)

In [42]:
print('Количество сочетаний "после":', count_stat(json_data))

Количество сочетаний "после": 26729228


In [43]:
unique_nouns = set()
for part in json_data:
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                unique_nouns.update(json_data[part][verb][adp][case])
print('Количество уникальных существительных "после":', len(unique_nouns))

Количество уникальных существительных "после": 45380


In [44]:
with open('./all_data_after_filter_2.json', 'w', encoding='UTF-8') as f:
    json.dump(json_data, f)

In [2]:
with open('./all_data_after_filter_2.json', 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

In [4]:
verb_adp_case = set()

for part in json_data:
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                temp = f"{verb}_{adp}_{case}"
                verb_adp_case.add(temp)
print('После фильтрации "глагол + предлог + падеж"', len(verb_adp_case))

После фильтрации "глагол + предлог + падеж" 340007


In [5]:
with open('./all_data.json', 'r', encoding='UTF-8') as f:
    json_data_1 = json.load(f)

In [6]:
verb_adp_case_1 = set()

for part in json_data_1:
    for verb in json_data_1[part]:
        for adp in json_data_1[part][verb]:
            for case in json_data_1[part][verb][adp]:
                temp = f"{verb}_{adp}_{case}"
                verb_adp_case_1.add(temp)
print('До фильтрации "глагол + предлог + падеж"', len(verb_adp_case_1))

До фильтрации "глагол + предлог + падеж" 466543


In [7]:
verb_adp_case_noun_1 = set()

for part in json_data_1:
    for verb in json_data_1[part]:
        for adp in json_data_1[part][verb]:
            for case in json_data_1[part][verb][adp]:
                for noun in json_data_1[part][verb][adp][case]:
                    temp = f"{verb}_{adp}_{case}_{noun}"
                    verb_adp_case_noun_1.add(temp)
print('До фильтрации "глагол + предлог + падеж + существительное"', len(verb_adp_case_noun_1))

До фильтрации "глагол + предлог + падеж + существительное" 9006225


In [8]:
verb_adp_case_noun = set()

for part in json_data:
    for verb in json_data[part]:
        for adp in json_data[part][verb]:
            for case in json_data[part][verb][adp]:
                for noun in json_data[part][verb][adp][case]:
                    temp = f"{verb}_{adp}_{case}_{noun}"
                    verb_adp_case_noun.add(temp)
print('После фильтрации "глагол + предлог + падеж + сущ"', len(verb_adp_case_noun))

После фильтрации "глагол + предлог + падеж + сущ" 6095208


In [12]:
count_stat(json_data_1['_'])

35591133

In [13]:
count_stat(json_data_1['не'])

2249948

In [14]:
count_stat(json_data_1)

37841081

In [16]:
count_stat(json_data['не'])

1573117

In [17]:
count_stat(json_data)

26729228

In [30]:
verb_not = set()
verb_not.update(json_data['не'].keys())

In [31]:
len(verb_not)

12684

In [32]:
verb = set()
verb.update(json_data['_'].keys())

In [33]:
len(verb)

21748

In [34]:
count = 0
for i in verb_not:
    if i not in verb:
        count += 1
print('Глаголы, использующиеся только с "не"', count)

Глаголы, использующиеся только с "не" 32


In [35]:
count = 0
for i in verb:
    if i not in verb_not:
        count += 1
print('Глаголы, использующиеся только без "не"', count)

Глаголы, использующиеся только без "не" 9096


In [36]:
count = 0
for i in verb:
    if i in verb_not:
        count += 1
print('Глаголы, использующиеся и с "не" и без "не"', count)

Глаголы, использующиеся и с "не" и без "не" 12652
